##### pip install azure-identity
##### pip install azure-keyvault-secrets

#### Key Vault Creation

##### az login
##### az keyvault create --name {your-unique-keyvault-name} --resource-group {resource group name}

Eg. az keyvault create --name testingAyanKeyVault --resource-group madasHealthChatBot

#### Role Assignment

#### az role assignment create --role "Key Vault Secrets Officer" --assignee "{upn}" --scope "/subscriptions/{subscription-id}/resourceGroups/{resource-group-name}/providers/Microsoft.KeyVault/vaults/{your-unique-keyvault-name}"

Replace {upn}, {subscription-id}, {resource-group-name} and {your-unique-keyvault-name} with your actual values. Your UPN will typically be in the format of an email address (e.g., username@domain.com).

In [2]:
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

os.environ["KEY_VAULT_NAME"] = "testingAyanKeyVault"

#### Pushing Into Key Vault

###### Cannot Run Again

In [8]:
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

secretName = "azure-openai-endpoint-value" # Name has to look like this  
secretValue = "https://sample-health-azureopenai-final.openai.azure.com/"
client.set_secret(secretName, secretValue) # SET Values

<KeyVaultSecret [https://testingayankeyvault.vault.azure.net/secrets/azure-openai-endpoint-value/9f049eb368cf4c07a17ca2dc83420ae3]>

#### Retrieve Secret

In [4]:
keyVaultName = os.environ["KEY_VAULT_NAME"]
KVUri = f"https://{keyVaultName}.vault.azure.net"

secretName = "azure-openai-endpoint-value" # Name has to look like this  

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

retrieved_secret = client.get_secret(secretName)
retrieved_secret

<KeyVaultSecret [https://testingayankeyvault.vault.azure.net/secrets/azure-openai-endpoint-value/9f049eb368cf4c07a17ca2dc83420ae3]>

In [5]:
retrieved_secret.value

'https://sample-health-azureopenai-final.openai.azure.com/'

#### Delete Secret

In [11]:
poller = client.begin_delete_secret(secretName)
deleted_secret = poller.result()

#### Recover Deleted Secret

In [12]:
client.begin_recover_deleted_secret(secretName).result()

<SecretProperties [https://testingayankeyvault.vault.azure.net/secrets/azure-openai-endpoint-value/9f049eb368cf4c07a17ca2dc83420ae3]>

#### SIngle-Ton

In [3]:
import os
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from threading import Lock

os.environ["KEY_VAULT_NAME"] = "testingAyanKeyVault"

class AzureKeyVaultClient:
    _instance = None
    _lock = Lock()

    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super(AzureKeyVaultClient, cls).__new__(cls)
                    cls._instance._initialize()
        return cls._instance

    def _initialize(self):
        key_vault_name = os.environ["KEY_VAULT_NAME"]
        kv_uri = f"https://{key_vault_name}.vault.azure.net"
        credential = DefaultAzureCredential()
        self.client = SecretClient(vault_url=kv_uri, credential=credential)

    def get_secret(self, secret_name):
        return self.client.get_secret(secret_name).value


In [4]:
kv_client = AzureKeyVaultClient()
openai_endpoint = kv_client.get_secret("azure-openai-endpoint-value")
openai_endpoint

'https://sample-health-azureopenai-final.openai.azure.com/'

In [5]:
kv_client_two = AzureKeyVaultClient()
openai_endpoint = kv_client_two.get_secret("azure-openai-endpoint-value")
openai_endpoint

'https://sample-health-azureopenai-final.openai.azure.com/'

In [6]:
print(f"kv1 id: {id(kv_client)}")
print(f"kv2 id: {id(kv_client_two)}")
print("Same instance?", kv_client is kv_client_two)

kv1 id: 1748841083920
kv2 id: 1748841083920
Same instance? True
